<a href="https://colab.research.google.com/github/neverlast121/Emotion-detection-using-Deepface/blob/main/CNN_IN_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# hyper _parameters
num_epoch = 10
batch_size = 4
learning_rate = 1e-3

# data tranformation
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

In [ ]:
# downloading data from torchvision datasets

train_data = datasets.CIFAR10(
    root ='data',
    train = True,
    download = True,
    transform = transform
)

test_data = datasets.CIFAR10(
    root = 'data',
    train = False,
    download = True,
    transform = transform

)

100%|██████████| 170498071/170498071 [00:05<00:00, 29024579.64it/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [ ]:
# dataloader

train_loader = DataLoader(
    dataset = train_data,
    batch_size = batch_size,
    shuffle = True

)

test_loader = DataLoader(
    dataset = test_data,
    batch_size = batch_size,
    shuffle = False
)

In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# ConvNet
class ConvNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Conv2d(3, 6, kernel_size = 5),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Conv2d(6, 16, kernel_size = 5),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Flatten(),
        nn.Linear(16*5*5, 120),
        nn.ReLU(),
        nn.Linear(120, 84),
        nn.ReLU(),
        nn.Linear(84, 10)
    )

  def forward(self, x):
    out = self.model(x)
    return out

model = ConvNet().to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
# training Loop
n_total_step = len(train_loader)
for epoch in range(num_epoch):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    # forward pass
    outputs = model(images)
    loss = loss_fn(outputs, labels)

    # backward optimizer
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 2000 == 0:
      print(f' Epoch [{epoch+1}/{num_epoch}], step [{i+1}/ {n_total_step}], loss: {loss.item():.4f}')
  print('**** Training Finish ****')

 Epoch [1/10], step [2000/ 12500], loss: 2.3559
 Epoch [1/10], step [4000/ 12500], loss: 2.3110
 Epoch [1/10], step [6000/ 12500], loss: 2.2759
 Epoch [1/10], step [8000/ 12500], loss: 2.3074
 Epoch [1/10], step [10000/ 12500], loss: 2.3008
 Epoch [1/10], step [12000/ 12500], loss: 2.2955
**** Training Finish ****
 Epoch [2/10], step [2000/ 12500], loss: 2.1442
 Epoch [2/10], step [4000/ 12500], loss: 2.1182
 Epoch [2/10], step [6000/ 12500], loss: 2.0751
 Epoch [2/10], step [8000/ 12500], loss: 1.7095
 Epoch [2/10], step [10000/ 12500], loss: 1.5448
 Epoch [2/10], step [12000/ 12500], loss: 1.8485
**** Training Finish ****
 Epoch [3/10], step [2000/ 12500], loss: 2.0874
 Epoch [3/10], step [4000/ 12500], loss: 1.6865
 Epoch [3/10], step [6000/ 12500], loss: 1.5529
 Epoch [3/10], step [8000/ 12500], loss: 1.5361
 Epoch [3/10], step [10000/ 12500], loss: 1.5658
 Epoch [3/10], step [12000/ 12500], loss: 1.9058
**** Training Finish ****
 Epoch [4/10], step [2000/ 12500], loss: 3.0201
 Epo

In [ ]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)

    # max return (value, index)
    _, predictions = torch.max(outputs, 1)
    n_samples += labels.size(0)
    n_correct += (predictions == labels).sum().item()

    for i in range(batch_size):
      label = labels[i]
      pred = predictions[i]
      if (label == pred):
        n_class_correct[label] += 1
      n_class_samples[label] += 1

  acc = 100.0 * n_correct/ n_samples
  print(f'Accuracy: {acc}%')

  for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]}: {acc}%')

Accuracy: 53.98%
Accuracy of plane: 69.3%
Accuracy of car: 77.8%
Accuracy of bird: 31.4%
Accuracy of cat: 31.4%
Accuracy of deer: 42.5%
Accuracy of dog: 30.4%
Accuracy of frog: 70.1%
Accuracy of horse: 78.5%
Accuracy of ship: 55.6%
Accuracy of truck: 52.8%
